In [21]:
from torchyolo import YoloHub
from ocsort.ocsort import OCSort


In [25]:
model = YoloHub(
    config_path="./default_config.yaml",
    model_type="yolov5",
    model_path="yolov5s.pt",
)

result = model.predict(source="./frames/frame0.jpg")

YOLOv5  2023-3-31 Python-3.8.16 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 270 layers, 7235389 parameters, 0 gradients
Adding AutoShape... 
  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [23]:


tracker = OCSort()
#dets = yl.detector(image)
online_targets = tracker.update(dets)

NameError: name 'yl' is not defined